Set up imports/paths

In [1]:
import numpy as np
from training_functions import *
data_dir = "C:/Users/2021l/Documents/UROP/data/"
requests_file = 'request_yamls/training.yaml'
saved_json = 'short.json'

Break up data into training/test

In [2]:
### Only run this cell if there is no training.json file saved ###
save_training_json(requests_file, data_dir, saved_json)

Extracting activity labels from the HDF5 file
Extracting myo-left data from the HDF5 file
myo-left Data:
  Shape (624467, 8)
None 1654639897.4104638 None 1654643708.4825573
Extracting myo-right data from the HDF5 file
myo-right Data:
  Shape (616471, 8)
1654639897.4104638 1654639897.400493 1654643708.4825573 1654643708.47957
Extracting tactile-glove-left data from the HDF5 file
tactile-glove-left Data:
  Shape (22615, 32, 32)
1654639897.4104638 1654639897.7586246 1654643708.47957 1654643708.5254169
Extracting tactile-glove-right data from the HDF5 file
tactile-glove-right Data:
  Shape (22741, 32, 32)
1654639897.7586246 1654639897.753641 1654643708.47957 1654643708.555328
Original Data:
  Shape (2,)
  Sampling rate: 0.00 Hz

Resampled Data:
  Shape (190537,)
  Sampling rate: 50.00 Hz

Extracting myo-left data from the HDF5 file
myo-left Data:
  Shape (624467, 8)

Original Data:
  Shape (624467, 8)
  Sampling rate: 163.86 Hz

Resampled Data:
  Shape (190537, 8)
  Sampling rate: 50.00 Hz

In [13]:
with open('short.json') as json_file:
    data = ujson.load(json_file)

corresponding_streams = [('myo-left/emg', 'tactile-glove-left/tactile_data')] #,('myo-right/emg', 'tactile-glove-right/tactile_data')]
window_time = 2
train_x, train_y, test_x, test_y = window_data(data, corresponding_streams, window_time)


start windowing
{'Slice a potato', 'Clean a plate with a towel'}
100
original shape (502, 8) (502, 32, 32)
    Clean a plate with a towel 0 5
original shape (402, 8) (402, 32, 32)
    Clean a plate with a towel 1 4
original shape (499, 8) (499, 32, 32)
    Clean a plate with a towel 2 4
total windows =  13
original shape (2772, 8) (2772, 32, 32)
    Peel a potato 0 27
original shape (2738, 8) (2738, 32, 32)
    Peel a potato 1 27
original shape (2126, 8) (2126, 32, 32)
    Peel a potato 2 21
total windows =  88
original shape (2706, 8) (2706, 32, 32)
    Peel a cucumber 0 27
original shape (2206, 8) (2206, 32, 32)
    Peel a cucumber 1 22
original shape (1977, 8) (1977, 32, 32)
    Peel a cucumber 2 19
total windows =  156
original shape (540, 8) (540, 32, 32)
    Clean a pan with a towel 0 5
original shape (615, 8) (615, 32, 32)
    Clean a pan with a towel 1 6
original shape (543, 8) (543, 32, 32)
    Clean a pan with a towel 2 5
total windows =  172
original shape (1886, 8) (1886, 3

In [14]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(286, 100, 8)
(286, 100, 32, 32)
(63, 100, 8)
(63, 100, 32, 32)


In [21]:
print(train_x[4])

[[ 5.          6.          8.         18.         34.         29.
  10.08210003  8.        ]
 [ 5.          6.          8.         18.         34.         30.
  11.          8.        ]
 [ 5.          6.          8.         17.         34.         31.36624976
  12.          8.        ]
 [ 5.          6.          7.         17.         34.04235774 32.04235774
  12.          8.        ]
 [ 5.          6.          7.         16.         35.         34.
  13.          8.        ]
 [ 6.          6.          7.         16.         35.         35.
  13.          8.39393359]
 [ 6.          6.          7.         15.         36.         36.
  14.          9.        ]
 [ 6.          7.          7.         15.         36.74507177 36.
  14.         10.        ]
 [ 7.          7.          7.         15.         37.         36.
  14.         10.35387167]
 [ 7.          7.          7.         15.         37.         36.
  14.         11.        ]
 [ 7.          8.          7.         15.         37.5

In [26]:
train_x = np.average(train_x, tuple(range(2, train_x.ndim)))
train_y = np.average(train_y, tuple(range(2, train_y.ndim)))
test_x = np.average(test_x, tuple(range(2, test_x.ndim)))
test_y = np.average(test_y, tuple(range(2, test_y.ndim)))
print(train_x[0])
print(train_y[0])

[11.32712919 10.73435955 10.24960374  9.75        9.375       8.81402939
  8.71446402  8.38705293  8.40162203  8.43290271  8.5         8.55020927
  8.82582801  9.02314593  9.33846326  9.75       10.01024224 10.02163637
 10.5        10.375      10.55915101 10.625      10.75       10.75
 10.75       10.57335704 10.5        10.5720789  10.4875925  10.43160885
 10.5        10.5        10.59184585 10.875      11.12735232 11.375
 11.375      11.875      11.93170782 12.10860594 12.07337654 11.89816764
 11.94710572 11.75       11.5        11.25       11.3592569  11.18706618
 11.20548083 11.2283926  10.97016787 11.17137663 11.125      11.348603
 11.375      11.25       11.5        11.875      11.875      12.
 12.125      12.1175441  12.09196736 12.0742185  12.23917342 11.81379469
 11.625      11.375      11.1041411  10.82530289 10.43911843 10.09823192
  9.51888426  8.875       8.625       8.375       7.89091851  7.625
  7.72151615  7.37759063  7.5592895   7.625       7.73746497  7.75
  7.984033

Create Model

In [ ]:
model = []

Training/Plotting loop

In [ ]:
# https://keras.io/guides/training_with_built_in_methods/
# maybe set aside validation samples?
history = model.fit(train_x, train_y, batch_size=100, epochs=2)

Evaluate trained model

In [ ]:
results = model.evaluate(test_x, test_y, batch_size=128)
print("test loss, test acc:", results)